<a href="https://www.kaggle.com/code/dikramasrour/preproc-modelling-kaggle-transe?scriptVersionId=116218708" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://colab.research.google.com/github/dikraMasrour/Breast_Cancer_Risk_Factor_Prediction_KG/blob/main/Preproc_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Breast cancer biomedical KG Preprocessing

In [1]:
%%capture
!pip install torchkge

In [2]:
%%capture
!conda install -y gdown

#### Mounting drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
!gdown --id 1-383Q6iPXXNr047ZCw7g6QjkDcbA8-B4
!gdown --id 13YNwGftMtzdb0dL2OC7n3gsGsCSCHFpq
!gdown --id 1-5-2u3gGWpHcjOeSBDsF5okGbAOzo6dq

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-383Q6iPXXNr047ZCw7g6QjkDcbA8-B4
To: /kaggle/working/test_df.csv
100%|████████████████████████████████████████| 156M/156M [00:11<00:00, 13.7MB/s]
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=13YNwGftMtzdb0dL2OC7n3gsGsCSCHFpq
To: /kaggle/working/train_df.csv
100%|████████████████████████████████████████| 271M/271M [00:16<00:00, 16.0MB/s]
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don

#### Imports

In [4]:
# basics
import pandas as pd
import numpy as np
from os.path import join
from os import makedirs, remove
from os.path import exists
from tqdm.autonotebook import tqdm
import pickle
import gc

# torchkge related
import torch
from torch import cuda
from torch.optim import Adam
from torchkge.models import TransEModel
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import MarginLoss, DataLoader
from torchkge.data_structures import KnowledgeGraph
from torchkge.utils import get_data_home
from torchkge.utils import Trainer
from torchkge.evaluation import LinkPredictionEvaluator

# Model path for colab
# MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/KG_breast_cancer/models'
# Model path for Kaggle
MODEL_PATH = '/kaggle/working/'

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [ ]:
# set up cuda if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# dataPath = "/content/drive/MyDrive/Colab Notebooks/KG_breast_cancer/preprocessed_KG.pkl"
# df = pd.read_pickle(dataPath)
# df.head()

In [ ]:
# df.shape

In [ ]:
# def split(df_):
#     # group by is used, so the model is able to train on all the nodes
#     train_df = df_.groupby(['from', 'to']).sample(frac = 0.8) # samples triples from each group
#     test_val = df_[~df_.index.isin(train_df.index)]
#     test_df = df_.sample(frac = 0.5)
#     val_df = test_val[~test_val.index.isin(test_df.index)]
#     print(train_df.shape, test_df.shape, val_df.shape)
#     return train_df, test_df, val_df


In [ ]:
# # RUN ONCE
# train_df, test_df, val_df = split(df)

In [ ]:
# # RUN ONCE : save splits as pickle files
# train_df.to_pickle('/content/drive/MyDrive/Colab Notebooks/KG_breast_cancer/splits/train_df.pkl')
# test_df.to_pickle('/content/drive/MyDrive/Colab Notebooks/KG_breast_cancer/splits/test_df.pkl')
# val_df.to_pickle('/content/drive/MyDrive/Colab Notebooks/KG_breast_cancer/splits/val_df.pkl')

In [5]:
gc.collect()

23

In [5]:
def load_kgs_from_df(data_home=None):
#     data_home = '/content/drive/MyDrive/Colab Notebooks/KG_breast_cancer/splits'
    data_home = '/kaggle/working'
    df1 = pd.read_csv(data_home + '/train_df.csv')
    df2 = pd.read_csv(data_home + '/val_df.csv')
    df3 = pd.read_csv(data_home + '/test_df.csv')
    df = pd.concat([df1, df2, df3])
    kg = KnowledgeGraph(df)

    return kg.split_kg(sizes=(len(df1), len(df2), len(df3)))

In [6]:
# RUN ONCE
kg_train, kg_val, kg_test = load_kgs_from_df()

In [7]:
gc.collect()

23

In [ ]:
# class TrainLoop():
#     def __init__(self, args , kg_train, kg_val, kg_test):
#         self.n_epochs = args.n_epochs
#         self.kg_train, self.kg_val, self.kg_test = kg_train, kg_val, kg_test
#         self.train_dataloader = DataLoader(kg_train, batch_size=args.b_size, use_cuda='all')
#         self.val_dataloader = DataLoader(kg_val, batch_size=args.b_size, use_cuda='all')
#         self.test_dataloader = DataLoader(kg_test, batch_size=args.b_size, use_cuda='all')
#         self.model_path = args.model_path
#         self.model = TransEModel(args.emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
#         self.criterion = MarginLoss(args.margin)
#         # Move everything to CUDA if available
#         if cuda.is_available():
#             cuda.empty_cache()
#             self.model.cuda()
#             self.criterion.cuda()

#         # Define the torch optimizer to be used
#         self.optimizer = Adam(self.model.parameters(), lr=args.lr, weight_decay=1e-5)

#         self.sampler = BernoulliNegativeSampler(self.kg_train)

#     def fit_step(self):
#         running_loss = 0.0
#         for i, batch in enumerate(self.train_dataloader):
# #             print('batch #', i)
#             h, t, r = batch[0].to(device), batch[1].to(device), batch[2].to(device)
#             n_h, n_t = self.sampler.corrupt_batch(h.to('cpu'), t.to('cpu'), r.to('cpu'))
#             n_h, n_t = n_h.to(device), n_t.to(device)
#             self.optimizer.zero_grad()

#             # forward + backward + optimize
#             pos, neg = self.model(h, t, r, n_h, n_t)
#             loss = self.criterion(pos, neg)
#             loss.backward()
#             self.optimizer.step()
# #             print(loss.item())
#             running_loss += loss.item()
#             gc.collect()
#         return running_loss
#     def test_step(self):
#         self.model = torch.load(self.model_path)
#         self.model.eval()
#         evaluator = LinkPredictionEvaluator(self.model, self.kg_test)
#         evaluator.evaluate(200, verbose=True)
#         evaluator.print_results()

#     def eval_step(self):
#         evaluator = LinkPredictionEvaluator(self.model, self.kg_val)
#         evaluator.evaluate(200, verbose=False)
#         return evaluator.mrr()[1]

#     def fit(self):
#         iterator = tqdm(range(self.n_epochs), unit='epoch')
#         best_val = -np.inf
#         patience= 10
#         patience_count = 0
#         for epoch in iterator:
#             running_loss = self.fit_step()
#             val_ = self.eval_step()

#             if val_> best_val:
#                 best_val = val_
#                 torch.save(self.model, self.model_path)
#                 patience_count = 0
#             else:
#                 if patience_count == patience:
#                     break
#                 else:
#                     patience_count += 1

#             iterator.set_description(
#                 'Epoch {} | mean loss: {:.5f}'.format(epoch + 1,
#                                                       running_loss / len(self.train_dataloader)))

#         self.model.normalize_parameters()

In [8]:
class Params():
    def __init__(self):
        pass

In [9]:
args = Params()

In [14]:
args.emb_dim = 100
args.lr = 0.0004
args.n_epochs = 1000
args.b_size = 32768
args.margin = 0.5
args.model_path = join(MODEL_PATH, "basic.bt")

In [ ]:
# tloop = TrainLoop(args, kg_train, kg_val, kg_test)

In [15]:
gc.collect()

46

In [ ]:
# tloop.fit()

In [16]:
model = TransEModel(args.emb_dim, kg_train.n_ent, kg_train.n_rel,
                    dissimilarity_type='L2')
criterion = MarginLoss(args.margin)
optimizer = Adam(model.parameters(), lr=args.lr, weight_decay=1e-5)
trainer = Trainer(model, criterion, kg_train, args.n_epochs, args.b_size,
                  optimizer=optimizer, sampling_type='bern',use_cuda='all')

In [17]:
trainer.run()

  0%|          | 0/1000 [00:00<?, ?epoch/s]

In [35]:
gc.collect()

23

In [36]:
evaluator = LinkPredictionEvaluator(model, kg_test)
evaluator.evaluate(15)
evaluator.print_results()

Link prediction evaluation:   0%|          | 0/275994 [00:00<?, ?batch/s]

Hit@10 : 0.111 		 Filt. Hit@10 : 0.171
Mean Rank : 951 	 Filt. Mean Rank : 677
MRR : 0.048 		 Filt. MRR : 0.077


In [ ]:
# tloop.test_step()